# simran4@wisc.edu
# rgundavarapu@wisc.edu

In [20]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['project-5-srh4dawin-db-1', 'project-5-srh4dawin-db-2', 'project-5-srh4dawin-db-3'])
    cass = cluster.connect()
    cass.execute("drop keyspace if exists weather")
except Exception as e:
    print(e)

In [21]:
import pandas as pd
pd.DataFrame(cass.execute("describe keyspaces"))

,keyspace_name,type,name
0,system,keyspace,system
1,system_auth,keyspace,system_auth
2,system_distributed,keyspace,system_distributed
3,system_schema,keyspace,system_schema
4,system_traces,keyspace,system_traces
5,system_views,keyspace,system_views
6,system_virtual_schema,keyspace,system_virtual_schema


# Part 1: Station Metadata

In [22]:
# DATA SETUP

cass.execute("""
CREATE KEYSPACE weather 
WITH REPLICATION = {
    'class' : 'SimpleStrategy', 
    'replication_factor' : 3 
    };
""")

cass.execute("""
CREATE TYPE 
    weather.station_record (tmin int, tmax int)
""")

cass.execute("""
CREATE TABLE weather.stations (
    id text,
    name text STATIC,
    date date,
    record weather.station_record,
    state text,
    PRIMARY KEY (id,date)
) WITH CLUSTERING ORDER BY (date asc)
""")

In [23]:
pd.DataFrame(cass.execute("describe keyspaces"))

,keyspace_name,type,name
0,system,keyspace,system
1,system_auth,keyspace,system_auth
2,system_distributed,keyspace,system_distributed
3,system_schema,keyspace,system_schema
4,system_traces,keyspace,system_traces
5,system_views,keyspace,system_views
6,system_virtual_schema,keyspace,system_virtual_schema
7,weather,keyspace,weather


# Q1

In [24]:
print(cass.execute("describe keyspace weather").one().create_statement)
print(cass.execute("describe table weather.stations").one().create_statement)

CREATE KEYSPACE weather WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}  AND durable_writes = true;
CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    state text,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair

In [25]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p5")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.2.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-44f64dc6-8fa3-45aa-bbfb-7879c669652e;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [26]:
!wget 'https://pages.cs.wisc.edu/~harter/cs639/data/ghcnd-stations.txt'
df = spark.read.text("ghcnd-stations.txt")

--2023-04-14 01:16:09--  https://pages.cs.wisc.edu/~harter/cs639/data/ghcnd-stations.txt
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10607756 (10M) [text/plain]
Saving to: ‘ghcnd-stations.txt’

ghcnd-stations.txt  100%[===================>]  10.12M  37.0MB/s    in 0.3s    

2023-04-14 01:16:10 (37.0 MB/s) - ‘ghcnd-stations.txt’ saved [10607756/10607756]



# Q2

In [ ]:
output= subprocess.check_output(["nodetool", "ring"])
output

# Part 2:Temperature Data